# Kor LLM 모델 서빙 (SageMaker Python SDK)

### 참조: 
- Model 정보
    - beomi/KoAlpaca-Polyglot-12.8B
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KoAlpaca Dataset v1.1b
        - https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B
    - EleutherAI/polyglot-ko-12.8b
        - Polyglot-Ko-12.8B was trained for 167 billion tokens over 301,000 steps on 256 A100 GPUs with the GPT-NeoX framework. It was trained as an autoregressive language model, using cross-entropy loss to maximize the likelihood of predicting the next token.
        - License: Apache 2.0
        - https://huggingface.co/EleutherAI/polyglot-ko-12.8b
        
- Doc
    - Large model inference tutorials
        - https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-tutorials.html
    - Use DJL with the SageMaker Python SDK
        - https://sagemaker.readthedocs.io/en/stable/frameworks/djl/using_djl.html
        
- 블로그
    - https://aws.amazon.com/ko/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/
    - 코드
        - https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb

# 1. 기본 환경 설정

In [1]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../common_code')

# 2. SageMaker endpoint 의 추론 도커 이미지 인 DLC image URL 가져오기
- We get DLC image URL for djl-deepspeed 0.21.0 and set SageMaker settings

In [2]:
import sagemaker, boto3
from sagemaker import image_uris


role = sagemaker.get_execution_role()  # execution role for the endpoint
session = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = session._region_name
bucket = session.default_bucket()  # bucket to house artifacts

img_uri = image_uris.retrieve(framework="djl-deepspeed", region=region, version="0.21.0")
img_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117'

# 3. Set configuration

## 테스트 모델 지정

In [3]:
serve_model = 'KoAlpaca-12-8B'
# serve_model = 'Polyglot-Kor-5-8B'

# 모델 패키징 할 파일 이름
model_artifact_name = f'{serve_model}.tar.gz'
# 모델 패키징 S3 위치
s3_location = f"s3://{bucket}/{serve_model}/"

print("serve_model: ", serve_model)
print("model_artifact_name: ", model_artifact_name)
print("model packaging s3 location: ", s3_location)


serve_model:  KoAlpaca-12-8B
model_artifact_name:  KoAlpaca-12-8B.tar.gz
model packaging s3 location:  s3://sagemaker-us-east-1-057716757052/KoAlpaca-12-8B/


## 로컬 모드 혹은 클라우드 모드

In [4]:
use_local_mode = False
# use_local_mode = True

if use_local_mode:
    instance_type = "local_gpu"
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()
    instance_type = "ml.g5.12xlarge"
    
print("instance_type  :", instance_type)      

instance_type  : ml.g5.12xlarge


# 4. 모델 추론 코드 및 모델 설정 파일을 패키징
- `model.py` and `serving.properties`
- The code below creates the SageMaker model file (`model.tar.gz`) and upload it to S3. 

In [5]:
%%sh -s {serve_model} {model_artifact_name}
serve_model=$1
model_artifact_name=$2
echo $serve_model
echo $model_artifact_name

rm -rf $serve_model/.ipynb_checkpoints

tar -czvf $model_artifact_name $serve_model/



KoAlpaca-12-8B
KoAlpaca-12-8B.tar.gz
KoAlpaca-12-8B/
KoAlpaca-12-8B/serving.properties
KoAlpaca-12-8B/model.py


## mode.tar.gz 를 S3 업로드

In [6]:
model_tar_url = sagemaker.s3.S3Uploader.upload(model_artifact_name, s3_location)
print("model_tar_url: ", model_tar_url)

model_tar_url:  s3://sagemaker-us-east-1-057716757052/KoAlpaca-12-8B//KoAlpaca-12-8B.tar.gz


# 5. SageMaker Model 생성

In [7]:
from sagemaker.model import Model
from sagemaker import image_uris, get_execution_role


def create_model(model_name, role, sagemaker_session, inference_image_uri, model_s3_url):
    model = Model(
        image_uri=inference_image_uri,
        model_data=model_s3_url,
        role=role,
        name=model_name,
        sagemaker_session=sagemaker_session,
    )
    return model

In [8]:
from datetime import datetime

time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model_name = f"{serve_model}-" + time_stamp

sm_model = create_model(model_name, role, sagemaker_session, img_uri, model_tar_url)

# 6. SageMaker Endpoint 생성
- 클라우드 배포시 약 8분 걸림. 10 이상 걸리면 무엇인가 문제 있음

In [9]:
from sagemaker import serializers, deserializers

def deploy_model(model, sagemaker_session, instance_type, _endpoint_name):
    model.deploy(
        initial_instance_count=1,
        instance_type="ml.g5.12xlarge",
        endpoint_name=_endpoint_name
    )
    predictor = sagemaker.Predictor(
        endpoint_name=_endpoint_name,
        sagemaker_session=sagemaker_session,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer()
    )
    return predictor

In [10]:
%%time

endpoint_name = f"{serve_model}-" + time_stamp
predictor = deploy_model(sm_model, sagemaker_session, instance_type, endpoint_name)


-------------!

# 7. 엔드포인트 추론 

In [11]:

from inference_lib import invoke_inference
    

## (1) 맥락 (Context) 없이 질문

In [12]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = ""#"홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다."
prompt_wo_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
print("prompt_wo_c: \n", prompt_wo_c)

prompt_wo_c: 
 ### 질문: 홈플러스 중계점은 몇시까지 장사해?

### 답변:


In [13]:
%%time 

invoke_inference(endpoint_name, prompt_wo_c)

### 질문: 홈플러스 중계점은 몇시까지 장사해?

### 답변:홈플러스 중계점은 오후 11시까지 영업합니다. 

홈플러스는 매장마다 조금씩 다르지만, 대부분의 매장이 오전 10시부터 오후 11시까지 운영됩니다. 따라서 홈플러스 중계점의 경우 오후 11시까지 장사를 합니다. 
CPU times: user 43.7 ms, sys: 20.7 ms, total: 64.5 ms
Wall time: 3.53 s


## (2) 맥락 (Context) 가지고 질문

In [14]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = "홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 "
prompt_w_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
print("prompt_w_c:\n", prompt_w_c)

prompt_w_c:
 ### 질문: 홈플러스 중계점은 몇시까지 장사해?

### 맥락: 홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 

### 답변:


In [15]:
%%time 

invoke_inference(endpoint_name,  prompt_w_c)

### 질문: 홈플러스 중계점은 몇시까지 장사해?

### 맥락: 홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 

### 답변:홈플러스의 영업시간은 오전 10시부터 오후 10시까지입니다. 

홈플러스의 홈페이지에서 영업점을 검색하여 해당 점포의 영업 시간을 확인할 수 있습니다. 휴무일과 다양한 점포 정보도 함께 알아보세요. 
CPU times: user 13 ms, sys: 3.07 ms, total: 16.1 ms
Wall time: 2.75 s


# 7. [중요] 클린업 엔트포인트 

In [16]:
# delete endpoint
predictor.delete_model()
predictor.delete_endpoint()

# Trouble Shooting

3vep2qi5ar-algo-1-zk8i8 | INFO  ModelServer Model server stopped.

3vep2qi5ar-algo-1-zk8i8 | ERROR ModelServer Invalid configuration: Workflow KoAlpaca_12_8B is already registered.